In [ ]:
import json
import os

import pandas as pd
from datasets_sota import DATASETS_SOTA, SotaMetricEnum
from IPython.display import display


def compare_to_sota(
        input_space_results,
        transfer_finetune_results,
    ) -> pd.DataFrame:

    sota_column = {}
    input_space_column = {}
    best_transfer_column = {}
    for dataset, (metric_enum, sota_value) in DATASETS_SOTA.items():
        sota_column[dataset] = sota_value
        if metric_enum == SotaMetricEnum.ACCURACY:
            input_space_column[dataset] = input_space_results.loc[dataset, "top1_acc"]
            best_transfer_column[dataset] = transfer_finetune_results.loc[dataset, "top1_acc"]
        elif metric_enum == SotaMetricEnum.WEIGHTED_F1_SCORE:
            input_space_column[dataset] = input_space_results.loc[dataset, "top1_f1"]
            best_transfer_column[dataset] = transfer_finetune_results.loc[dataset, "top1_f1"]
    sota_column = pd.Series(sota_column, name="SOTA")
    input_space_column = pd.Series(input_space_column, name="Input Space")
    best_transfer_column = pd.Series(best_transfer_column, name="Best Transfer")
    df = pd.DataFrame({
        "SOTA": sota_column,
        "Input Space": input_space_column,
        "Input Space Delta": (input_space_column - sota_column).round(2),
        "Best Transfer (Fine-Tuned Model)": best_transfer_column,
        "Best Transfer Delta": (best_transfer_column - sota_column).round(2),
    })
    return df

def compare_transfer_methods(
        from_scratch_results,
        transfer_lp_results,
        transfer_knn_results,
        transfer_finetune_results,
        ) -> pd.DataFrame:
    df = pd.DataFrame({
        "From Scratch": from_scratch_results["top1_acc"],
        "Linear Probing": transfer_lp_results["top1_acc"],
        "Linear Probing Delta": (transfer_lp_results["top1_acc"] - from_scratch_results["top1_acc"]).round(2),
        "k-NN": transfer_knn_results["top1_acc"],
        "k-NN Delta": (transfer_knn_results["top1_acc"] - from_scratch_results["top1_acc"]).round(2),
        "Fine-Tuned Model": transfer_finetune_results["top1_acc"],
        "Fine-Tuned Model Delta": (transfer_finetune_results["top1_acc"] - from_scratch_results["top1_acc"]).round(2),
    })
    return df

RESULTS_PATH = "./final-results"
per_dataset_input_space = {}
per_dataset_from_scratch = {}
per_dataset_transfer_knn = {}
per_dataset_transfer_lp = {}
per_dataset_transfer_finetune = {}

for dataset_name in DATASETS_SOTA.keys():
    input_space_path = os.path.join(RESULTS_PATH, "input-space", f"{dataset_name}-INPUT_SPACE.json")
    from_scratch_path = os.path.join(RESULTS_PATH, "from-scratch", f"{dataset_name}-FROM_SCRATCH.json")
    transfer_knn_path = os.path.join(RESULTS_PATH, "transfer-knn", f"{dataset_name}-KNN.json")
    transfer_lp_path = os.path.join(RESULTS_PATH, "transfer-linear-probe", f"{dataset_name}-LINEAR_PROBE.json")
    transfer_finetune_path = os.path.join(RESULTS_PATH, "transfer-finetune", f"{dataset_name}-FINETUNE.json")
    if os.path.exists(input_space_path):
        with open(input_space_path) as f:
            per_dataset_input_space[dataset_name] = json.load(f)["test_metrics"]
    if os.path.exists(from_scratch_path):
        with open(from_scratch_path) as f:
            per_dataset_from_scratch[dataset_name] = json.load(f)["test_metrics"]
    if os.path.exists(transfer_knn_path):
        with open(transfer_knn_path) as f:
            per_dataset_transfer_knn[dataset_name] = json.load(f)["test_metrics"]
    if os.path.exists(transfer_lp_path):
        with open(transfer_lp_path) as f:
            per_dataset_transfer_lp[dataset_name] = json.load(f)["test_metrics"]
    if os.path.exists(transfer_finetune_path):
        with open(transfer_finetune_path) as f:
            per_dataset_transfer_finetune[dataset_name] = json.load(f)["test_metrics"]

input_space_results = (pd.DataFrame.from_dict(per_dataset_input_space, orient="index") * 100).round(2).rename_axis("Input Space Baseline")
from_scratch_results = (pd.DataFrame.from_dict(per_dataset_from_scratch, orient="index") * 100).round(2).rename_axis("From Scratch")
transfer_lp_results = (pd.DataFrame.from_dict(per_dataset_transfer_lp, orient="index") * 100).round(2).rename_axis("Transfer Linear Probe")
transfer_knn_results = (pd.DataFrame.from_dict(per_dataset_transfer_knn, orient="index") * 100).round(2).rename_axis("Transfer k-NN")
transfer_finetune_results = (pd.DataFrame.from_dict(per_dataset_transfer_finetune, orient="index") * 100).round(2).rename_axis("Transfer Fine-Tuned Model")

df_compare_methods = compare_transfer_methods(
    from_scratch_results,
    transfer_lp_results,
    transfer_knn_results,
    transfer_finetune_results,
)

df_compare_sota = compare_to_sota(
    input_space_results,
    transfer_finetune_results,
)

# display(input_space_results)
# display(from_scratch_results)
# display(transfer_knn_results)
# display(transfer_lp_results)
# display(transfer_finetune_results)

#### Table 6
Results for Table 6 of the manuscript, comparing transfer methods with training from scratch.

In [18]:
display(df_compare_methods)

,From Scratch,Linear Probe,Linear Probe Delta,k-NN,k-NN Delta,Fine-Tuned Model,Fine-Tuned Model Delta
ISCXVPN2016-App,75.71,72.81,-2.90,73.25,-2.46,77.44,1.73
ISCXVPN2016-TrafficType,78.35,73.01,-5.34,75.57,-2.78,80.19,1.84
ISCXVPN2016-Encapsulation,93.96,88.03,-5.93,91.83,-2.13,94.37,0.41
MIRAGE19,82.49,72.51,-9.98,84.75,2.26,85.31,2.82
MIRAGE22,98.48,95.89,-2.59,98.19,-0.29,98.67,0.19
UTMOBILENET21,85.30,86.43,1.13,86.85,1.55,88.90,3.60
UCDAVIS19-Script,98.73,98.00,-0.73,100.00,1.27,98.13,-0.60
UCDAVIS19-Human,78.07,87.47,9.40,84.82,6.75,89.64,11.57
CESNET-TLS22,98.03,85.65,-12.38,96.44,-1.59,98.25,0.22
AppClassNet,92.42,62.15,-30.27,81.12,-11.30,92.11,-0.31


#### Table 7
Results for Table 7 of the manuscript, comparing the fine-tuning transfer method to SOTA and the input-space baseline.

In [20]:
display(df_compare_sota)

,SOTA,Input Space,Input Space Delta,Best Transfer (Fine-Tuned Model),Best Transfer Delta
ISCXVPN2016-App,63.92,70.91,6.99,77.44,13.52
ISCXVPN2016-TrafficType,65.56,73.01,7.45,80.19,14.63
ISCXVPN2016-Encapsulation,85.45,90.38,4.93,94.37,8.92
MIRAGE19,80.06,79.93,-0.13,85.30,5.24
MIRAGE22,97.18,95.63,-1.55,98.67,1.49
UTMOBILENET21,81.91,83.86,1.95,88.73,6.82
UCDAVIS19-Script,98.63,97.87,-0.76,98.13,-0.50
UCDAVIS19-Human,80.45,72.77,-7.68,89.64,9.19
CESNET-TLS22,97.20,90.95,-6.25,98.25,1.05
AppClassNet,88.30,76.20,-12.10,92.11,3.81
